<a href="https://colab.research.google.com/github/loudly-soft/experiments/blob/main/MultiDoc_Summarisation_PivotedQR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Benchmark Text Summarisation with Rouge

My self-learning notes on how to use Rouge to benchmark text summarisation and where to get ground truth data to test your model.

**Synopsis**

1. Download news summary dataset
2. Implement pivoted QR algorithm [1] to summarize news from multiple sources
3. Calculate precision, recall and F1-score with Rouge

**Catalog of NLP Datasets**
* https://metatext.io/datasets-list/summarization-task

**Rouge Tutorials**
* https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460
* https://kavita-ganesan.com/what-is-rouge-and-how-it-works-for-evaluation-of-summaries/#.YVcnVZ1KiUk

**References**

[1] Conroy J, O’Leary D., Text Summarization via Hidden Markov Models and Pivoted QR Matrix Decomposition.  Technical Report, University of Maryland, College Park, Maryland, March, 2001.

## Download ground truth data for text summarisation

I will be using the multi-document summarisation dataset from:

* https://github.com/Alex-Fabbri/Multi-News

Each news summary is generated from multiple source articles.  There is a preprocessed and an unprocessed version of the data.  Both versions are divided into train, test and validation dataset with summaries and source articles.

For this demo, I'll work with the pre-processed version of validation data:
* https://drive.google.com/drive/folders/1uDarzpu2HFc-vjXNJCRv2NIHzakpSGOw


In [ ]:
!gdown --id 1fpLqEb4lQ2F0ooBzyBoVc-d2S1qh-euS
!gdown --id 1RzmVVqVMNWhjNTUWKeiBS-HW1UIqnXeS

Downloading...
From: https://drive.google.com/uc?id=1fpLqEb4lQ2F0ooBzyBoVc-d2S1qh-euS
To: /content/val.txt.tgt
7.55MB [00:00, 117MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RzmVVqVMNWhjNTUWKeiBS-HW1UIqnXeS
To: /content/val.txt.src
62.8MB [00:00, 151MB/s]


The files are:
* `val.txt.tgt` - summaries
* `val.txt.src` - source articles

Count number of lines in files for sanity check

In [ ]:
!wc -l val.txt.tgt
!wc -l val.txt.src

5622 val.txt.tgt
5622 val.txt.src


#### Disable horizontal scroll bar so long output in Colab is wrapped



In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''<style>pre { white-space: pre-wrap; }</style>'''))
get_ipython().events.register('pre_run_cell', set_css)

#### Extract source articles and summaries

The source file `val.txt.src` has multiple stories per line delimited by *'story_separator_special_tag'*

In [ ]:
multi_sources = []

with open('val.txt.src', 'r', encoding='UTF-8') as file:
  for line in file:
    articles = line.strip().split('story_separator_special_tag')
    multi_sources.append(sorted([text.strip() for text in articles if text], key=len))

# show the first source stories
for i, article in enumerate(multi_sources[0]):
  print('Source %d of %d' % (i+1, len(multi_sources[0])))
  print('--------------')
  print(article.replace('\n', ' '))
  print()
  

Source 1 of 4
--------------
a charity shop is urging people to stop donating the da vinci code after becoming overwhelmed with copies .     the oxfam shop in swansea has been receiving an average of one copy of the dan brown novel a week for months , leaving them with little room for any other books .     staff who are struggling to sell copies of the book have put a note up in the store saying they would rather donors hand in their vinyl instead .

Source 2 of 4
--------------
whether a sign of a good read ; or a comment on the ' pulp ' nature of some genres of fiction , the oxfam second-hand book charts have remained in the da vinci code author ' s favour for the past four years .     dan brown has topped oxfam ' s ' most donated ' list again , his fourth consecutive year . having sold more than 80 million copies of the da vinci code and had all four of his novels on the new york times bestseller list in the same week , it ' s hardly surprising that brown ' s hefty tomes are being d

The summary file `val.txt.tgt` has 1 summary per line

In [ ]:
summaries = []
with open('val.txt.tgt', 'r', encoding='UTF-8') as file:
  for line in file:
    summaries.append(line.rstrip()[2:])

# show the first summary
print(summaries[0])


the da vinci code has sold so many copies — that would be at least 80 million — that it ' s bound to turn up in book donation piles . but at one charity shop in the uk , it ' s been donated so heavily that the shop has posted a sign propped up on a tower of da vinci code copies that reads : " you could give us another da vinci code ... but we would rather have your vinyl ! " the manager of the oxfam shop in swansea tells the telegraph that people are laughing and taking pictures of the sizable display : " i would say that we get one copy of the book every day . " he says people buy them " occasionally , " but with vinyl sales up 25 % in the past year , they ' d rather take records . dan brown ' s book isn ' t the only one that shops like oxfam struggle to re-sell . last year , oxfam was hit with a large and steady supply of fifty shades of grey , and it similarly begged donors : " please — no more . " but brown has a particular kind of staying power . the da vinci code was published in

## Generate extractive summaries

The algorithm is QR decomposition with column pivoting from numerical linear algebra [1].  The algorithm selects key sentences and ranks them in order of importance.  The idea is to find the most important sentence (biggest L2 norm) and extract it for summary.  From the remaining sentences, extract the next sentence that is important and different (orthogonal) to the extracted ones.  Repeat until enough key sentences are extracted.

I'm curious how this approach will fare on multi-document summary.  Would it be able to ignore conceptually similar sentences from different sources when the sources are combined into a single text for summarisation?

**References**

[1] Conroy J, O’Leary D., Text Summarization via Hidden Markov Models and Pivoted QR Matrix Decomposition.  Technical Report, University of Maryland, College Park, Maryland, March, 2001.


#### Implement pivoted QR decomposition

In [ ]:
from numpy import linalg as LA
import numpy as np


def givens_vector(x, start_pos, eps=0.0000001):
  """Apply givens rotation to zero out elements of vector x from starting position"""

  size = len(x)
  G_acc = np.eye(size)
  for i in range(size-1, start_pos-1, -1):
    if x[i-1] < eps and x[i] < eps:
      continue
    # init givens matrix
    c = x[i-1] / pow(pow(x[i-1], 2) + pow(x[i], 2), 0.5)
    s = x[i] / pow(pow(x[i-1], 2) + pow(x[i], 2), 0.5)
    G = np.eye(size)
    G[i-1, i-1] = c
    G[i-1, i] = s 
    G[i, i-1] = -s
    G[i, i] = c
    x = G.dot(x)
    # stack the givens matrices
    G_acc = np.matmul(G, G_acc)
  return G_acc, x


def extract_columns(W, max_cols, seed_col_index=None, eps=0.0000001):
  """Extract orthogonal columns with biggest L2 norm"""

  selected_col_indices = []
  col_indices = list(range(W.shape[1]))
  while len(selected_col_indices) < max_cols and len(col_indices) > 0:
    l2_norms = LA.norm(W, axis=0)
    if seed_col_index is None:
      # pick column of W with highest L2 norm
      j = np.argmax(l2_norms)
    else:
      j = seed_col_index
      seed_col_index = None
    if l2_norms[j] < eps:
      break
    selected_col_indices.append(col_indices[j])
    del col_indices[j]
    # apply givens rotation to selected column
    G, _ = givens_vector(W[:,j], len(selected_col_indices))
    # apply givens rotation to remaining columns
    W = np.matmul(G, np.delete(W, j, 1))
  return selected_col_indices


A simple test to show that the heaviest (biggest L2 norm) column is chosen first, then the next heaviest column orthogonal to the chosen column is selected, then the next column orthogonal to all the chosen columns.

In [ ]:
X = np.array([
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 1, 0],
 [0, 1, 0],
 [1, 0, 0]])

extract_columns(X, max_cols=3)

[2, 1, 0]

#### Functions to vectorize text to term-sentence matrix

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')


def vectorize(sentences, truncateSVD=True, threshold=0.8):
  """Convert sentences to term-sentence matrix and reduce noise with SVD"""

  vectorizer = CountVectorizer(stop_words="english", strip_accents='ascii')
  W = vectorizer.fit_transform(sentences).toarray().transpose().astype(float)
  
  # truncate SVD to cover X% of singular values
  if truncateSVD:
    U, S, V = np.linalg.svd(W, full_matrices=True)
    total_sigmas = sum(S)
    subtotal = 0
    for k, s in enumerate(S):
      subtotal += s
      if subtotal / total_sigmas > threshold:
        break
    W = V[0:k+1,:]
    #print('%d out of %d sigmas selected' % (k+1, len(S)))
  return W


def parse_sentences(text):
  """Tokenize text into sentences"""

  return [sentence.replace('\n', ' ') for sentence in nltk.tokenize.sent_tokenize(text)]


def summarise(W, sentences, max_sentences, seed_col_index=None):
  """Apply pivoted QR on term-sentence matrix"""

  # sort extracted sentences by their original position in text
  return ' '.join([sentences[i] for i in sorted(extract_columns(W, max_sentences, seed_col_index))])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Apply algorithm to summarise news

In [ ]:
from tqdm.notebook import tqdm


summaries_test = []
summaries_pred = []

for i in tqdm(range(len(summaries))):
#for i in tqdm(range(100)):

  # tokenize sentences from source articles and combine them
  sentences = sum([parse_sentences(text) for text in multi_sources[i]], [])

  # pivoted QR algo is slow so skip long text
  if len(sentences) <= 200:

    # choose only the longest source article to summarise and tokenize sentences
    #sentences = parse_sentences(multi_sources[i][-1])

    # vectorize sentences to term-sentence matrix
    W = vectorize(sentences, truncateSVD=True, threshold=0.2)

    # set max number of summary sentences to % of the longest article
    #max_summary_sentences = max(1, int(len(parse_sentences(multi_sources[i][-1])) * 0.20))

    # set max number of summary sentences
    max_summary_sentences = 6

    # generate summary
    summary = summarise(W, sentences, max_summary_sentences)
    summaries_pred.append(summary)

    # keep ground truth summary
    summaries_test.append(summaries[i])

print(f'{len(summaries_pred)} out of {len(summaries)} summaries generated')


  0%|          | 0/5622 [00:00<?, ?it/s]

5307 out of 5622 summaries generated


Show the first summary generated by model

In [ ]:
print(summaries_pred[0])

a woman reads a copy of the newly released book ' ' the lost symbol ' ' by dan brown , at a speed reading book launch event in sydney , september 15 , 2009. reuters/tim wimborne     san francisco the latest novel from " da vinci code " author dan brown , " the lost symbol , " broke one-day sales records , its publisher and booksellers said . readers snapped up over one million hardcover copies across the united states , canada and the united kingdom after it was released on tuesday , said publisher knopf doubleday , a division of random house inc.     " we are seeing historic , record-breaking sales across all types of our accounts in north america for ' the lost symbol , " said sonny mehta , editor in chief of knopf doubleday publishing group . but as secondhand bookshop shelves flood with battered editions of angels and demons and digital fortress , brown can comfort himself with the fact that he ' s also oxfam ' s second most bought author : there are , apparently , still readers ou

## Benchmark with Rouge
Install `rouge` library from https://github.com/pltrdy/rouge

In [ ]:
!pip install rouge

#### Calculate precision, recall and F1-score

In [ ]:
from rouge import Rouge

# metric 'route-l' (with an L at the end) is slow, so ignore
rouge = Rouge(metrics=['rouge-1', 'rouge-2'])
scores = rouge.get_scores(summaries_pred, summaries_test, avg=True)

# print rouge metrics
metric_descs = {'f':'f1-score', 'p':'precision', 'r':'recall'}
for score_name, metrics in scores.items():
  print(score_name.upper())
  print('-' * 2 * len(score_name))
  for metric_name, val in metrics.items():
    print(f'{metric_descs[metric_name]} = {val}')
  print()


ROUGE-1
--------------
recall = 0.3839146727188993
precision = 0.3379472602044078
f1-score = 0.350632333558966

ROUGE-2
--------------
recall = 0.13253740035283365
precision = 0.11745861062942464
f1-score = 0.12041542842694532

